In [ ]:
#using Revise
using DrWatson
@quickactivate "ABoxWorld"
include(srcdir("ABoxWorld.jl"));

In [1]:
using LinearAlgebra
using Symbolics

In [2]:
function double_noisy_PRBox(;q1, q2)
    @assert 0.0 - 10*eps() <= q1 <= 1.0 + 10*eps() 
    @assert 0.0 - 10*eps() <= q2 <= 1.0 + 10*eps()
    @assert q1 + q2 <= 1.0 + 100*eps()
    @show q1, q2, 1.0-q1-q2
    return (1.0-q1-q2)*CanonicalPRBox() + q1*LocalDeterministicBoxesCHSH(α=1, γ=0, β=0, λ=0) + q2*LocalDeterministicBoxesCHSH(α=1, γ=0, β=1, λ=0)
    #return c2*extremal_noise_generator(;extremal_noise_params...) + (1-c1-c2)*UniformRandomBox(scenario)
end

evaluate(Original_IC_Bound(), double_noisy_PRBox(q1=0.3, q2=0.05))

(q1, q2, (1.0 - q1) - q2) = (0.3, 0.05, 0.6499999999999999)


4.929999999999999

In [3]:



function exp_channels(noisy_PRBox_generator)

    boundary_params = Dict()

    #eval_conditions = Dict("Pawlowski et al., 2009" => Original_IC_Bound(), "Jain et al., 2023" => Correlated_Inputs_IC_Bound(ϵ=0.99),"Raw: Z-Channel, p=0.25" => ZChannel_Raw_IC_Bound(;n=2, e_c=-0.49), "Raw: Z-Channel, p=0.0" => ZChannel_Raw_IC_Bound(;n=2, e_c=-0.99)) #"Raw: Jain et al., 2023" => Jain_Raw_IC_Bound(;n=2, e_c=0.01), ,"Raw: Erasure Channel" => Erasure_Raw_IC_Bound(;n=2, e_η=0.01),
    eval_conditions = Dict("Pawlowski et al., 2009" => Original_IC_Bound(),"Jain et al., 2023" => Correlated_Inputs_IC_Bound(ϵ=0.99),"Raw: Z-Channel, p=0.8" => ZChannel_Raw_IC_Bound(;n=2, e_c=0.6)) #"Raw: Z-Channel, p=0.25" => ZChannel_Raw_IC_Bound(;n=2, e_c=-0.49))  #"Raw: Erasure Channel" => Erasure_Raw_IC_Bound(;n=2, e_η=0.99)

    for c_condition_key in keys(eval_conditions)
        boundary_params[c_condition_key] = []

        q1_range::Vector{Union{Float64, Missing}} = collect(0.0:0.001:1.0)
        q2_stepsize = 1/4000
        q2_boundary::Vector{Union{Float64, Missing}} = []
        
        for q1 in q1_range
            
            boundary_q2_val = 0.0  # Find largest q2 such that the box is still in Q

            #First check whether we are inside Q already
            if check(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1), :Q)
                boundary_q2_val = missing
            #Next, check whether we are outside, but nevertheless already very very close to Q's boundary (essentially on it), otherwise start search for the boundary
            elseif abs(evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                while !check(eval_conditions[c_condition_key], noisy_PRBox_generator(q2=boundary_q2_val, q1=q1), :Q)

                    boundary_q2_val += q2_stepsize
                    
                    if boundary_q2_val > 1.0 - q1 + 10*eps() #Can't go beyond 1.0 - q1 = no valid convex mixture
                        
                        boundary_q2_val -= q2_stepsize #Go back one step

                        break
                    end
                end
            end
            push!(boundary_params[c_condition_key], (q1,boundary_q2_val))
            
        end

    end
    return eval_conditions, boundary_params
end



exp_channels (generic function with 1 method)

In [4]:
using GLMakie

eval_conditions, boundary_params = exp_channels(double_noisy_PRBox)
@show boundary_params
cmap = [:gold, :deepskyblue3, :orangered, "#e820b6", :darkgreen]

with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do

    fig = Figure(; size = (800,600), fontsize = 22)
    ax = Axis(fig[1, 1]; title = " ", xgridstyle = :dash, ygridstyle = :dash, xlabel = "q1", ylabel = "q2")
        
    for condition_key in keys(eval_conditions)
        q1_vals, q2_vals = map(q -> q[1], filter(q -> !ismissing(q[2]), boundary_params[condition_key])), map(q -> q[2], filter(q -> !ismissing(q[2]), boundary_params[condition_key]))
        #lines!(q1_vals, q2_vals; label = condition_key) #, color = :green
        
        band!(q1_vals, q2_vals, 1 .- q1_vals; label = condition_key)
    end
    
    #translate!(lineplot, 0, 0, 2) # move line to foreground
    limits!(-0.001, 1.001, 0.001, 0.301)
    axislegend(; position = :ct, backgroundcolor = (:white, 0.75), framecolor = :orange)
    save("./assets/q1q2_channels.png", fig, size = (800, 600))
    fig
end

(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0, 1.0)


(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0, 1.0)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0, 1.0)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.00025, 0.99975)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0005, 0.9995)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.00075, 0.99925)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.001, 0.999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.00125, 0.99875)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0015, 0.9985)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.00175, 0.99825)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.002, 0.998)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0022500000000000003, 0.99775)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0025000000000000005, 0.9975)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0027500000000000007, 0.99725)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.003000000000000001, 0.997)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.003250000000000001, 0.99675)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0035000000000000014, 0.9965)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.0037500000000000016, 0.99625)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.004000000000000


(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04000000000000003, 0.96)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04025000000000003, 0.95975)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04050000000000003, 0.9595)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04075000000000003, 0.9592499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04100000000000003, 0.959)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04125000000000003, 0.95875)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04150000000000003, 0.9585)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04175000000000003, 0.9582499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04200000000000003, 0.958)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04225000000000003, 0.95775)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04250000000000003, 0.9575)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04275000000000003, 0.9572499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04300000000000003, 0.957)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04325000000000003, 0.95675)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.04350000000000003, 0.9565)
(q1, q2, (1.0 - q1)

(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07800000000000006, 0.9219999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07825000000000006, 0.92175)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07850000000000006, 0.9215)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07875000000000006, 0.9212499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07900000000000006, 0.9209999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07925000000000006, 0.92075)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07950000000000006, 0.9205)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.07975000000000006, 0.9202499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08000000000000006, 0.9199999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08025000000000006, 0.91975)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08050000000000006, 0.9195)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08075000000000006, 0.9192499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08100000000000006, 0.9189999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.08125000000000006, 0.91875)
(q1, q2, (1.0 - q1) - q2) = (0

(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11425000000000009, 0.8857499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11450000000000009, 0.8855)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11475000000000009, 0.8852499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11500000000000009, 0.8849999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11525000000000009, 0.8847499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11550000000000009, 0.8845)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11575000000000009, 0.8842499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11600000000000009, 0.8839999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11625000000000009, 0.8837499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11650000000000009, 0.8835)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11675000000000009, 0.8832499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11700000000000009, 0.8829999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11725000000000009, 0.8827499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.0, 0.11750000000000009, 0.8825

...

(q1, q2, (1.0 - q1) - q2) = (0.008, 0.034750000000000024, 0.9572499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.035000000000000024, 0.957)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.035250000000000024, 0.95675)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.035500000000000025, 0.9565)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.035750000000000025, 0.9562499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.036000000000000025, 0.956)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.036250000000000025, 0.95575)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.036500000000000025, 0.9555)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.036750000000000026, 0.9552499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.037000000000000026, 0.955)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.037250000000000026, 0.95475)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.037500000000000026, 0.9545)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.03775000000000003, 0.9542499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.03800000000000003, 0.954)
(q1, q2, (1.0 - q1) - q2) = (0.

(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07150000000000005, 0.9205)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07175000000000005, 0.9202499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07200000000000005, 0.9199999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07225000000000005, 0.91975)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07250000000000005, 0.9195)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07275000000000005, 0.9192499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07300000000000005, 0.9189999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07325000000000005, 0.91875)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07350000000000005, 0.9185)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07375000000000005, 0.9182499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07400000000000005, 0.9179999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07425000000000005, 0.91775)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07450000000000005, 0.9175)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.07475000000000005, 0.9172499999999999)
(q1

(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10675000000000008, 0.8852499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10700000000000008, 0.8849999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10725000000000008, 0.8847499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10750000000000008, 0.8845)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10775000000000008, 0.8842499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10800000000000008, 0.8839999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10825000000000008, 0.8837499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10850000000000008, 0.8835)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10875000000000008, 0.8832499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10900000000000008, 0.8829999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10925000000000008, 0.8827499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10950000000000008, 0.8825)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.10975000000000008, 0.8822499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008,

Excessive output truncated after 524308 bytes.

(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1410000000000001, 0.8509999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1412500000000001, 0.8507499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1415000000000001, 0.8504999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1417500000000001, 0.85025)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1420000000000001, 0.8499999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1422500000000001, 0.8497499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1425000000000001, 0.8494999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1427500000000001, 0.8492499999999998)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1430000000000001, 0.8489999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1432500000000001, 0.8487499999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1435000000000001, 0.8484999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1437500000000001, 0.84825)
(q1, q2, (1.0 - q1) - q2) = (0.008, 0.1440000000000001, 0.8479999999999999)
(q1, q2, (1.0 - q1) - q2) = (0.008

In [5]:
@show boundary_params

Dict{Any, Any} with 3 entries:
  "Raw: Z-Channel, p=0.8"  => Any[(0.0, 0.20775), (0.001, 0.20725), (0.002, 0.2…
  "Pawlowski et al., 2009" => Any[(0.0, 0.2), (0.001, 0.19975), (0.002, 0.19925…
  "Jain et al., 2023"      => Any[(0.0, 0.24975), (0.001, 0.2495), (0.002, 0.24…

In [10]:


function getNoisyPRBoxGenerator(extremal_noise_generator; extremal_noise_params...)
    scenario = (2,2,2,2)
    @assert extremal_noise_generator ∈ [LocalDeterministicBoxesCHSH, PRBoxesCHSH]
    function noisy_PRBox_generator(;c1::Float64, c2::Float64)
        #@show typeof(c1), typeof(c2), typeof(c1 + c2)
        @assert 0.0 - 10*eps() <= c1 <= 1.0 + 10*eps() "Got c1=$c1, but c1 must be in [0,1]!"
        @assert 0.0 - 10*eps() <= c2 <= 1.0 + 10*eps() "Got c2=$c2, but c2 must be in [0,1]!"
        @assert c1 + c2 <= 1.0 + 100*eps() "Got c1+c2=$(c1+c2), but c1 + c2 should not exceed 1.0!"
        return c1*CanonicalPRBox() + c2*extremal_noise_generator(;extremal_noise_params...) + (1.0-c1-c2)*UniformRandomBox(scenario)
        #return c2*extremal_noise_generator(;extremal_noise_params...) + (1-c1-c2)*UniformRandomBox(scenario)
    end
    return noisy_PRBox_generator
end

function exp1channels(noisy_PRBox_generatorm, first_chsh, second_chsh)

    boundary_params = Dict()
    CHSH_CHSH2_boundary_vals = Dict()

    eval_conditions = Dict("IC Orig." => Original_IC_Bound(), "IC Corr. Inputs" => Correlated_Inputs_IC_Bound(ϵ=0.99), "IC Z-Channel, p=0.75" => ZChannel_Raw_IC_Bound(;n=2, e_c=0.49),"IC Z-Channel, p=0.5" => ZChannel_Raw_IC_Bound(;n=2, e_c=0.01)) #"IC Z-Channel, p=0.25" => ZChannel_Raw_IC_Bound(;n=2, e_c=-0.49))  #

    for c_condition_key in keys(eval_conditions)
        boundary_params[c_condition_key] = []
        CHSH_CHSH2_boundary_vals[c_condition_key] = []

        c2_range::Vector{Union{Float64, Missing}} = collect(0.0:0.01:1.0)
        c1_stepsize = 1/3000
        c1_boundary::Vector{Union{Float64, Missing}} = []
        
        for c2 in c2_range
            
            boundary_c1_val = 0.0  # Find largest c1 such that the box is still in Q
            made_steps = 1 #Keep track of floating point precision on recurrent subtraction

            #First check whether we are outside Q already
            if !check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)
                boundary_c1_val = missing
            #Next, check whether we are inside, but nevertheless already very very close to Q's boundary (essentially on it), otherwise start search for the boundary
            elseif abs(evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                while check(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2), :Q)

                    boundary_c1_val += c1_stepsize
                    made_steps += 1
                    if boundary_c1_val > 1.0 - c2 + 10*eps() #Can't go beyond 1.0 - c2 = no valid convex mixture
                        #println(evaluate(CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), " ",evaluate(CHSH_Inequality(), noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)))
                        
                        boundary_c1_val -= c1_stepsize #Go back one step
                        
                        #Check whether we, nevertheless, are at the boundary
                        """Uncomment when want to label positivity boundaries as missing
                        if abs(evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)) - eval_conditions[c_condition_key].setBounds[:Q]) > 10e3*eps() 
                            #@show "Here", boundary_c1_val, c2
                            #@show evaluate(eval_conditions[c_condition_key], noisy_PRBox_generator(c1=boundary_c1_val, c2=c2)), eval_conditions[c_condition_key].setBounds[:Q]
                            boundary_c1_val = missing
                        end
                        """

                        break
                    end
                end
            end
            push!(boundary_params[c_condition_key], (boundary_c1_val, c2))
            
            if ismissing(boundary_c1_val)
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (missing, missing))
            else
                CHSH_val = evaluate(first_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                CHSH2_val = evaluate(second_chsh, noisy_PRBox_generator(c1=boundary_c1_val, c2=c2))
                push!(CHSH_CHSH2_boundary_vals[c_condition_key], (CHSH_val, CHSH2_val))
            end
        end

    end
    return eval_conditions, boundary_params, CHSH_CHSH2_boundary_vals
end


noisy_PRBox_generator = getNoisyPRBoxGenerator(LocalDeterministicBoxesCHSH; 
                                                α=0, γ=0, β=0, λ=0)

eval_conditions,boundary_params, CHSH_CHSH2_boundary_vals = exp1channels(noisy_PRBox_generator,
                                                CHSH_Inequality(),
                                                CHSH_Inequality(s1=1, s2=-1, s3=1, s4=1),
                                                )

cmap = [:deepskyblue3, :orangered, :gold, "#e82051"]


with_theme(palette = (; patchcolor = cgrad(cmap, alpha=0.45))) do
    fig3 = lines(collect(0.0:0.01:2.0), 4.0 .- collect(0.0:0.01:2.0); label = "N.S.", color=:black,
    axis = (; xlabel = "CHSH2", ylabel = "CHSH", title ="Title"),
    figure = (; size = (800,600), fontsize = 22))

    for condition_key in keys(eval_conditions)
        tlm_chsh, tlm_chsh2 = map(x -> x[1], CHSH_CHSH2_boundary_vals[condition_key]), map(x -> x[2], CHSH_CHSH2_boundary_vals[condition_key])
        lines!(tlm_chsh2, tlm_chsh; label = condition_key) #, color = :green
    end
    
    limits!(-0.001, 2.001, 2.001, 4.001)
    axislegend(; position = :ct, backgroundcolor = (:white, 0.75), framecolor = :orange)

    save("./assets/Channels_CHSH_vs_CHSH2_LD0000.png", fig3)
    fig3
end

In [11]:
@show CHSH_CHSH2_boundary_vals

Dict{Any, Any} with 4 entries:
  "IC Corr. Inputs"      => Any[(3.96133, 0.0), (3.94267, 0.02), (3.92267, 0.04…
  "IC Z-Channel, p=0.5"  => Any[(1.86133, 0.0), (1.86133, 0.02), (1.86133, 0.04…
  "IC Orig."             => Any[(2.82933, 0.0), (2.82933, 0.02), (2.82933, 0.04…
  "IC Z-Channel, p=0.75" => Any[(2.48933, 0.0), (2.48933, 0.02), (2.48933, 0.04…